### 堆, 可以理解为一个深度为$k$的二叉树, 在前$k-1$个深度上起都是完全二叉树
- 最大堆: 所有父节点大于孩子节点
- 最小堆: 所有父节点小于孩子节点
### 建堆
#### 从数组中构建堆
- n个元素的堆一定有$n-\lfloor \frac{n}{2} \rfloor$个叶节点, 参见联系6.1-7
因此从数组第$\lfloor \frac{n}{2} \rfloor$个元素开始, 往前遍历

#### 从插入中构建
- 和上述没有区别, 不过是每次插入节点都会调用heapify函数

In [1]:
def _left(idx):
    # if idx:
    return 2*idx
    # else:
        # return 1
def _right(idx):
    # if idx:
    return 2*idx + 1
    # else:
        # return 2

class Node:
    def __init__(self, key=0, pnt=None, cld=None, degree=0, mark=False):
        self.key = key
        # self.value = 0
        self.pnt = pnt
        self.cld = cld
        self.left = self
        self.right = self
        self.degree = degree
        self.mark = mark

    def __repr__(self):
        return str("key:{}, left:{}, right:{}".format(self.key,self.left.key,self.right.key))

class MinHeap:
    def __init__(self):
        self.nodes = [-1]
        self.length = 0

    def _heapify(self,idx):
        if idx > len(self.nodes):
            print("Heap Overflow!!")
            raise IndexError
        node = self.nodes[idx]
        idx_left = _left(idx)
        idx_right = _right(idx)

        # left = self.nodes[idx_left]
        # right = self.nodes[idx_right]
        
        idx_min = idx

        if idx_left < self.length and node.key > self.nodes[idx_left].key:
            idx_min = idx_left
        if idx_right < self.length and self.nodes[idx_min].key > self.nodes[idx_right].key:
            idx_min = idx_right
        
        if idx_min != idx:
            self.nodes[idx], self.nodes[idx_min] = self.nodes[idx_min], self.nodes[idx]
            self._heapify(idx_min)
    
    def _extractMin(self):
        result = self.nodes[1]
        if self.length > 0:
            print('before:{}'.format(self.nodes))
            self.nodes[1], self.nodes[self.length] = self.nodes[self.length], self.nodes[1]
            print("heap:{}".format(self.nodes))
            self._heapify(1)
            print('after:{}'.format(self.nodes))
            self.length -= 1
            return result
        else:
            print('underflow!')
    
    def _getMin(self):
        return self.nodes[1]
    
    def insert(self,node):
        self.nodes.insert(1,node)
        self.length += 1
        self._heapify(1)

斐波那契堆, 不会

In [33]:
import math
def _append(dest,node):
    """
    将node节点添加到dest节点左边
    """
    if not dest:
        print("dest is empty!")

    target = dest.left
    dest.left = node
    node.right = dest

    target.right = node
    node.left = target

def _concat(node1,node2):
    if not node1:
        print("node #1 empty!")
    if not node2:
        print("node #2 empty!")

    left_1 = node1.left
    left_2 = node2.left

    node1.left = left_2
    left_2.right = node1
    
    node2.left = left_1
    left_1.right = node2

def _delete(node):
    if not node:
        print("node is empty!")
    left = node.left
    right = node.right
    left.right = right
    right.left = left

    # node.left = None
    # node.right = None

def _link(y, x):
    """
    从根链表删除y, 把y加入x的孩子链表, x的度数+1且y的标志记为False
    """
    _delete(y)
    if x.cld:
        _append(x.cld,y)
    else:
        y.left = y
        y.right = y
        x.cld = y
    x.degree += 1
    y.mark = False

In [50]:
class Fibonacci_Heap():
    def __init__(self):
        self.min = None
        self.n = 0

    def union(self,H2):
        H = Fibonacci_Heap()
        H.min = self.min
        # 两者链表合并
        _concat(H.min, H2.min)

        if self.min == None or (H2.min != None and H2.min.key < self.min.key):
            H.min = H2.min
        
        H.n = self.n + H2.n
        return H
    
    def insert(self,x):
        x.degree = 0
        x.pnt = None
        x.cld = None
        x.mark = False
        if not self.min:
            x.left = x
            x.right = x
            self.min = x
        
        else:
            """
            加入到链表里, 放在min节点的左边, 根链表中节点次序是任意的, 所以直接加到最小节点的左边
            """
            _append(self.min,x)

            if x.key < self.min.key:
                self.min = x
        self.n += 1
    
    def extractMin(self):
        ptr = self.min
        if ptr:
            if ptr.cld:
                cld = ptr.cld
                start = cld

                while 1:
                    cld.prn = None
                    next_cld = cld.left

                    # ptr.left == ptr怎么办？
                    # 试了一下是没有影响的
                    _append(ptr.left,cld)
                    cld = next_cld
                    if cld == start:
                        break

                # _concat(ptr.left,ptr.cld)
                
            # 从根链表中删除ptr
            _delete(ptr)

            if ptr == ptr.right:
                self.min = None
            else:
                # 这里是随便指的
                self.min = ptr.right

                self._consolidate()

            self.n -= 1
        return ptr
    
    def _consolidate(self):
        A = [None] * int(math.log(self.n) + 1)
        
        start = self.min
        iterator = start
        while 1:
            degree = iterator.degree
            
            next_node = iterator.left
            
            while A[degree]:
                another = A[degree]
                
                if iterator.key > another.key:
                    iterator, another = another,iterator

                _link(another,iterator)

                A[degree] = None
                degree += 1
            
            A[degree] = iterator
            
            iterator = next_node
            if iterator == start:
                break
        
        self.min = None
        
        for node in A:
            if node:
                if self.min == None:
                    node.left = node
                    node.right = node
                    self.min = node
                else:
                    _append(self.min,node)
                    if node.key < self.min.key:
                        self.min = node

In [51]:
a = Node(1)
b = Node(2)
c = Node(3)
d = Node(4)
e = Node(5)

In [52]:
fib_heap_1 = Fibonacci_Heap()
fib_heap_2 = Fibonacci_Heap()
fib_heap_1.insert(d)
fib_heap_1.insert(a)
fib_heap_1.insert(c)
fib_heap_2.insert(b)
fib_heap_2.insert(e)

In [53]:
fib_heap_1.extractMin()

key:1, left:3, right:4

In [54]:
fib_heap_union = fib_heap_1.union(fib_heap_2)
fib_heap_union.extractMin()

key:2, left:3, right:5

In [12]:
fib_heap.extractMin()

key:4, left:4, right:4

In [13]:
fib_heap.min